# Aprendizado Supervisionado no Neurônio Perceptron

In [ ]:
# Importando as ferramentas

import numpy as np
import matplotlib.pyplot as plt
import random as rd
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score
import math
from prettytable import PrettyTable

## Identificador da Equipe

O identificador trata-se de um código numérico de um dígito gerado pela soma do último dígito do número de matrícula de cada integrante do projeto em módulo 4. Este será usado em especificidades do presente projeto. Dessa forma, os números de matrículas são:

* 2315080005
* 2315080061
* 2315080010
* 2315080021
* 2315080022

Portanto, o identificador é dado por (5+1+0+1+2)mod(4) = 1.

In [ ]:
ID = 1

##  Parte I– Resolvendo um Problema Linearmente Separável


1. Usar o arquivo dataAll.txt
2. Construir o algoritmo de treinamento do neurônio perceptron.

In [ ]:
arq_input = "dataAll.txt"

dataAll = np.fromfile(arq_input, dtype=np.float64)
print("Tipo de dataAll: {}".format(type(dataAll)))
print("Número de elementos em dataAll: {}".format(dataAll.size))
print("\n")


# -1 infers the size of the new dimension from the size of the input array.
dataAll = np.reshape(dataAll, (1000, 1, 3))
print(dataAll[:10]) # Mostrando 10 exemplos

print("\n")
print("Shape de dataAll: {}".format(dataAll.shape))
print("Dimensões de dataAll: {}".format(dataAll.ndim))

Como havia 3000 exemplos anteriormente e agora a matriz tem dimensões (1, 3). 
Então, há 3000/3 = 1000 exemplos em dataAll. Ou seja, (1000, 1, 3).

In [ ]:
# Separando em x, y e rotulo 

print("x:")
x = dataAll[:, :, :2]
print(x.shape)
print(x[:10]) # Mostrando os 10 primeiros pontos 
print("\n")

print("y:")
y = dataAll[:,:, 2:]
print(y.shape)
print(y[:10]) # Rótulos dos pontos

Criando o neurônio Perceptron:

In [ ]:
class Perceptron:
    """
    A perceptron is a classification model that consists of a set of weights, or scores,
    one for every feature, and a threshold. The perceptron multiplies each weight by its
    corresponding score, and adds them, obtaining a score.
    by Rajwrite Nath on Medium
    
    """

    def __init__(self, threshold, lr):
        self.threshold = threshold
        self.lr = lr
        self.weights = None
        self.scores = None
        self.adjustments = 0
        self.epochs = 0
        self.bias = -1

    def print_weights(self):
        np.set_printoptions()
        print("Weights:{}".format(self.weights))

    def gen_weights(self, low, high, size):
        self.weights = np.random.uniform(low, high, size=size).reshape(-1, 1)

    def step_fn(self, z):
        return np.where(z >= self.threshold, 1, 0)

    def error_fn(self, y_true, y_predicted):
        return y_true - y_predicted

    def predict(self, x_data):
        return self.step_fn(np.dot(x_data, self.weights))

    def fit(self, x_data, y_data):
        
        # Adicionando um bias ao x_data:
        bias = np.full((x_data.shape[0], 1, 1), self.bias)
        x_data = np.concatenate([bias, x_data], axis=2)
        print(x_data)
        print(x_data.shape)
        
        # Gerando pesos
        self.gen_weights(-0.5, 0.5, size=x_data.shape[2])
        print("Início:")
        self.print_weights()
        print(self.weights.shape)
        
        
        y_predicted = self.predict(x_data)
        self.scores = y_predicted  # Atualiza os scores

        # Algoritmo executa até a convergência, supomos que as classes sejam linearmente separáveis
        while not np.all(y_data == self.scores):

            self.epochs += 1  # Não convergiu, vai necessitar de mais 1 época

            # Para os pesos que resultaram em uma predição errada:

            predicoes_incorretas = np.where(y_data != self.scores)[0]

            for i in predicoes_incorretas:
                x_i = x_data[i]
                y_i = y_data[i]
                y_pred = y_predicted[i]

                # Calculando erros
                error = self.error_fn(y_i, y_pred)

                # Relculando pesos
                self.weights += self.lr * error * x_i.T
                self.adjustments += 1

            y_predicted = self.predict(x_data)
            self.scores = y_predicted # Atualiza os scores novamente com base nos ajustes

# Inicializando o neurônio
neuronio_perceptron = Perceptron(threshold=0, lr=0.1)
neuronio_perceptron.fit(x, y)
print("Final: ")
neuronio_perceptron.print_weights()

In [ ]:
# Gerando o gráfico

def gen_grafico(data, final_w, title):
    x_1 = data[:,:, :1]
    x_2 = data[:,:, 1:2]
    y = data[:, :, 2:]
    fig, ax = plt.subplots()
    cor = {0: 'red', 1:'blue'}
    for i in range(data.shape[0]):
        plt.plot(x_1[i][0][0], x_2[i][0][0], marker='o', c = cor[y[i][0][0]])
        
    
    x = np.arange(-1000, 1001, 1)
    reta = (final_w[0][0]/final_w[2][0]) - (final_w[1][0]/final_w[2][0]) * x
    plt.plot(x, reta, color='purple', linestyle='--', markersize=0.5)
    ax.legend(cor, loc='center left', bbox_to_anchor=(1, 0.5))
    ax.grid(True)
    ax.set_title(title)
    plt.show()

gen_grafico(dataAll, neuronio_perceptron.weights, "Distribução dos exemplos em dataAll")

# Parte III – Validação Holdout em Problema Não-Linearmente Separável

In [ ]:
arq_holdout = "dataHoldout.txt"

dataHoldout = np.fromfile(arq_holdout, dtype=np.float64)

print("Tipo de dataHoldout: {}".format(type(dataHoldout)))
print("Número de elementos em dataHoldout: {}".format(dataHoldout.size))
print("\n")

dataHoldout = np.reshape(dataHoldout, (800, 1, 3))
print(dataHoldout[:10])

print("\n")
print("Shape de dataHoldout: {}".format(dataHoldout.shape))
print("Dimensões de dataHoldout: {}".format(dataHoldout.ndim))

In [ ]:
def holdout_grafico(data,title):
    x_1 = data[:,:, :1]
    x_2 = data[:,:, 1:2]
    y = data[:, :, 2:]
    fig, ax = plt.subplots()
    cor = { 0: 'red', 1:'blue'}
    for i in range(data.shape[0]):
        plt.plot(x_1[i][0][0], x_2[i][0][0], marker='o', c = cor[y[i][0][0]])
    
    ax.legend(cor, loc='center left', bbox_to_anchor=(1, 0.5), labelcolor = ["red", "blue"])
    ax.set_title(title)
    plt.show()
        
holdout_grafico(dataHoldout,"Gráfico Inicial de Espalhamento")

In [ ]:
#Declarando a semente de aleatoriedade
np.random.seed(ID)

#Utilização do permutation para embaralhar a array dataHoldout
dataHoldout = np.random.permutation(dataHoldout)

#Definição do tamanho do array de treino como 70% do original
training_amount=int(800*0.7)

#Divisão de teste e treino
training_sample=dataHoldout[:(training_amount)]
test_sample=dataHoldout[(training_amount):]

#Análise se as amostras se mantém constantes 
print(training_sample[:10])
print("\n")
print(test_sample[:10])


In [ ]:
# Separando em x, y e rotulo de teste e treino

print("x_Treino:")
x_training= training_sample[:, :, :2]
print(x_training.shape)
print(x_training[:5]) # Mostrando os 5 primeiros pontos 
print("\n")

print("y_training:")
y_training = training_sample[:,:, 2:]
print(y_training.shape)
print(y_training[:5]) # Rótulos dos pontos


print("x_Teste:")
x_test= test_sample[:, :, :2]
print(x_test.shape)
print(x_test[:5]) # Mostrando os 5 primeiros pontos 
print("\n")

print("y_test:")
y_test = test_sample[:,:, 2:]
print(y_test.shape)
print(y_test[:5]) # Rótulos dos pontos

In [ ]:
class Controled_Perceptron:
    """
    This class works equals to the Perceptron class, but it doesn't stop when the neuron is all trained but when a
    limit of epochs has been passed
 
    """

    def __init__(self, threshold, lr ,limit):
        self.threshold = threshold
        self.lr = lr
        self.weights = None
        self.scores = None
        self.adjustments = 0
        self.epochs = 0
        self.bias = -1
        self.limit=limit

    def print_weights(self):
        np.set_printoptions()
        print("Weights:{}".format(self.weights))

    def gen_weights(self, low, high, size):
        self.weights = np.random.uniform(low, high, size=size).reshape(-1, 1)

    def step_fn(self, z):
        return np.where(z >= self.threshold, 1, 0)

    def error_fn(self, y_true, y_predicted):
        return y_true - y_predicted

    def predict(self, x_data):
        return self.step_fn(np.dot(x_data, self.weights))

    def fit(self, x_data, y_data):
        
        # Adicionando um bias ao x_data:
        bias = np.full((x_data.shape[0], 1, 1), self.bias)
        x_data = np.concatenate([bias, x_data], axis=2)
        print(x_data)
        print(x_data.shape)
        
        # Gerando pesos
        self.gen_weights(-0.5, 0.5, size=x_data.shape[2])
        print("Início:")
        self.print_weights()
        print(self.weights.shape)
        
        
        y_predicted = self.predict(x_data)
        self.scores = y_predicted  # Atualiza os scores

        # Algoritmo executa até a convergência, supomos que as classes sejam linearmente separáveis
        while(self.epochs<=self.limit):

            self.epochs += 1  # Não convergiu, vai necessitar de mais 1 época

            # Para os pesos que resultaram em uma predição errada:

            predicoes_incorretas = np.where(y_data != self.scores)[0]

            for i in predicoes_incorretas:
                x_i = x_data[i]
                y_i = y_data[i]
                y_pred = y_predicted[i]

                # Calculando erros
                error = self.error_fn(y_i, y_pred)

                # Relculando pesos
                self.weights += self.lr * error * x_i.T
                self.adjustments += 1

            y_predicted = self.predict(x_data)
            self.scores = y_predicted # Atualiza os scores novamente com base nos ajustes
            
neuronio_perceptron_holdout = Controled_Perceptron(threshold=0, lr=0.1, limit=100)
neuronio_perceptron_holdout.fit(x_training, y_training)
print("Final: ")
neuronio_perceptron_holdout.print_weights()

In [ ]:
def holdout_line_grafico(data,final_w,title):
    x_1 = data[:,:, :1]
    x_2 = data[:,:, 1:2]
    y = data[:, :, 2:]

    fig, ax = plt.subplots()
    cor = { 0: 'red', 1:'blue'}
    for i in range(data.shape[0]):
        plt.plot(x_1[i][0][0], x_2[i][0][0], marker='o', c = cor[y[i][0][0]])
        
    x = np.arange(-1, 3, 1)
    reta = (final_w[0][0]/final_w[2][0]) - (final_w[1][0]/final_w[2][0]) * x
    plt.plot(x, reta, color='purple', linestyle='--', markersize=0.5)
    
    ax.legend(cor, loc='center left', bbox_to_anchor=(1, 0.5))
    ax.set_title(title)
    plt.show()
        
holdout_line_grafico(dataHoldout,neuronio_perceptron_holdout.weights,"Distruibuição de exemplos de dataHoldout")

In [ ]:
def pred(data, neuron):
    bias = np.full((data.shape[0], 1, 1), neuron.bias)
    data = np.concatenate([bias, data], axis=2)
    return neuron.predict(data).reshape(-1)

y_training_pred = pred(x_training, neuronio_perceptron_holdout)
y_training = y_training.reshape(-1)

y_test_pred = pred(x_test, neuronio_perceptron_holdout)
y_test = y_test.reshape(-1)

cm_training = confusion_matrix(y_training, y_training_pred, labels=[0, 1])
cm_test = confusion_matrix(y_test, y_test_pred, labels=[0, 1])

# Plotando
fig, ax = plt.subplots(1, 2, figsize=(8, 4))

# Matriz de Treinamento
ConfusionMatrixDisplay(confusion_matrix=cm_training, display_labels=[0,1]).plot(ax=ax[0], cmap="Reds")
ax[0].set_title("Matriz de Confusão - Treinamento")

# Matriz de Teste
ConfusionMatrixDisplay(confusion_matrix=cm_test, display_labels=[0,1]).plot(ax=ax[1], cmap="Blues")
ax[1].set_title("Matriz de Confusão - Teste")

plt.tight_layout()
plt.show()


Diante da matriz de confusão obtida, percebe-se um comportamento parecido tanto para os dados de treinamento quanto para os de teste. Errando apenas em alguns casos, tendo valores de falsos positivos e falsos negativos aproximados para essas quantidades de dados. No entanto, uma análise mais detalhada é possível apenas mediante o cálculo das métricas, o que será realizado em seguinda.

Métricas a serem calculadas:
1. Acurácia
2. Precisão
3. Revocação
4. F-Score.

In [ ]:
training_metrics = {
    "accuracy" : accuracy_score(y_training, y_training_pred),
    "precision" : precision_score(y_training, y_training_pred, average="binary"),
    "recall" : recall_score(y_training, y_training_pred, average="binary"),
    "f_1Score" : f1_score(y_training, y_training_pred, average="binary")
    }

test_metrics = {
    "accuracy" : accuracy_score(y_test, y_test_pred),
    "precision" : precision_score(y_test, y_test_pred, average="binary"),
    "recall" : recall_score(y_test, y_test_pred, average="binary"),
    "f_1Score" : f1_score(y_test, y_test_pred, average="binary")
    }

table = PrettyTable()
table.padding_width = 8

table.title = "Métricas de treinamento e de teste"
table.field_names = ["", "Treinamento", "Teste"]
table.add_row(["Acurácia", f"{training_metrics["accuracy"]:.4f}", f"{test_metrics["accuracy"]:.4f}"])
table.add_row(["Precisão", f"{training_metrics["precision"]:.4f}", f"{test_metrics["precision"]:.4f}"])
table.add_row(["Revocação", f"{training_metrics["recall"]:.4f}", f"{test_metrics["recall"]:.4f}"])
table.add_row(["F1_Score", f"{training_metrics["f_1Score"]:.4f}", f"{test_metrics["f_1Score"]:.4f}"])
print(table)

In [ ]:
print(0.9138-0.9474/0.9474)

As métricas utilizaram a média (average) binária, dado que a tarefa é de classificação binária desbalanceada. Diante disso, as métricas obtidas foram elevadas, i.e. acima de 90% em geral. A perda percentual da métrica F1_Score foi relativamente pequena, com um decréscimo de 8,62%. Tais métricas indicam uma boa convergência do modelo, apesar de este procurar uma solução linear para um problema não linear. Contudo, em se tratando da aplicabilidade do modelo, não é possível definí-lo como eficiente apenas com os dados obtidos. Isso se deve ao fato do número de dados utilizados ser reduzido, de modo que seja possível não estar refletindo precisamente os outliers, tornando a predição pouco eficaz para possíveis outros datasets. 

In [ ]:
x = np.arange(-1, 3, 1)

w0, w1, w2 = neuronio_perceptron_holdout.weights
w0, w1, w2 = w0[0], w1[0], w2[0]

reta = (w0/w2) - (w1/w2) * x

fig, ax = plt.subplots(1, 2, figsize=(16, 8))
ax[0].plot(x, reta, color='purple', linestyle='--', markersize=0.5)
ax[1].plot(x, reta, color='purple', linestyle='--', markersize=0.5)

cor = {0 : "red", 1 : "blue"}

for i in range(x_training.shape[0]):
    ax[0].plot(x_training[i][0][0], x_training[i][0][1], marker='o', c = cor[y_training[i]])
ax[0].set_title("Distribuição de Dados para Treinamento")

for i in range(x_test.shape[0]):
    ax[1].plot(x_test[i][0][0], x_test[i][0][1], marker='o', c = cor[y_test[i]])
ax[1].set_title("Distribuição de Dados para Teste")

ax[1].legend(("Função de ativação", "0", "1"), loc='lower left', bbox_to_anchor=(1, 0), labelcolor="linecolor")


plt.show()

Com estes gráficos, é possível a visualização dos pontos dados como falsos positivos e falsos negativos na matriz de confusão, também é expressa a manutenção da proporção entre classes no processo de permutação do dataset durante o holdout.